# Reads

In [ ]:
https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right

a figura com drvs justifica muito bem a escolha de 80% para teste


fazer eval em todos dfs de 80%, adicionar epoch e indices etc para depois pegar caso necessario
pegar melhor modelo olhando o loss e aplicar pra todos circuitos de teste fcn, ou fcn_large



Figura:
como alguem faria para usar o preditor
como alguem faria para testar com outra tecnologia o preditor
explicar que ele he independente de tecnologia
fluxograma mostrando a parte do ORDF e a parte do INNOVUS para geracao dos benchamrks
dps para geracao dos dados de treino e inferencia.
Explicar o problema dos dados, não é só uma questão de armazenamento, mas a leitura deles
tbm seria muito demorada inviabilizando o treino do modelo.
justificar o porque de pegar soh short

# Imports

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import shutil
import os
import time

# Training Result

In [ ]:
# Training performace
# trainResultDF = pickle.load(open('cnn/trainResultDF.pkl', 'rb'))
trainResultDF = pickle.load(open('fcn/trainResultDF.pkl', 'rb'))
epochs = {x for x in trainResultDF['epoch']}
print('epoch, tp, tn, fp, fn')
for x in range(len(epochs)):
  epochDF = trainResultDF.loc[trainResultDF['epoch'] == x]
  print(x, epochDF['tp'].sum(), epochDF['tn'].sum(), epochDF['fp'].sum(), epochDF['fn'].sum(), sep=',')

In [ ]:
# Plot training metrics for a given design
# TODO: Maybe change this to take the average for all types of the same design
trainResultDF = pickle.load(open('fcn/trainResultDF.pkl', 'rb'))
design = 'swerv_84'
sortedDF = trainResultDF.loc[trainResultDF['design'] == design].sort_values(by=['epoch'])
ytrain = [x for x in sortedDF['tp']]
yval = [x for x in sortedDF['tn']]
plt.plot(ytrain, label = "tp")
plt.plot(yval, label = "tn")
plt.legend()
plt.title(design)
plt.show()

# Cross Validation Result

In [ ]:
crossValResultDF = pickle.load(open('fcnCV18.pkl', 'rb'))
rows = []
for index, row in crossValResultDF.iterrows():
  tp = row['tp']
  tn = row['tn']
  fp = row['fp']
  fn = row['fn']
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  TotalPos = tp+fn
  TotalNeg = fp+tn
  PosRatio = ((tp+fn)/(tp+fn+fp+tn))*100
  tpr = tp/(tp+fn)
  spc = tn/(tn+fp)
  fa = fp/(tn+fp)
  acc = (tp+tn)/(tp+tn+fp+fn)
  sqrt = math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
  mcc = 0
  if sqrt != 0:
    mcc = ((tp * tn) - (fp * fn))/sqrt
  rows.append({'tp':tp, 'tn':tn, 'fp':fp, 'fn':fn,
               'PosRatio%':PosRatio,
               'tpr':tpr, 'spc':spc, 'acc':acc, 'MCC[-1:1]':mcc})
  print()


crossValResultDF = pd.DataFrame(rows, index=crossValResultDF['Design'])
crossValResultDF.loc['mean'] = crossValResultDF.mean()
# crossValResultDF.to_csv('fcnCV18.csv')

# Benchmark Information Plot

In [ ]:
df = pd.read_pickle('benchmark.pkl', compression='zip')

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
numIDRShorts = dfRuntime['IDRShort']
dfRuntime = dfRuntime[['GR', 'IDR', 'FDR']]
ax = dfRuntime.plot.bar(stacked=True)

numBars = int(len(ax.patches)/3)
for rect, value in zip(ax.patches[numBars:numBars*2], numIDRShorts):
  h = rect.get_height() /2.
  w = rect.get_width() /2.
  x, y = rect.get_xy()
  ax.text(x+w, y+h, value, horizontalalignment='center',verticalalignment='center')

ax.set_ylabel('Runtime in seconds')
ax.set_xlabel('Designs (80% row utilization)')
ax.legend(["Global Routing", "Initial Detailed Routing", "Complete Detailed Routing"])
plt.title('Routing Runtime Break Down For 80% Row Utilization Designs')
# plt.savefig('routing_runtime.pdf')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
idrShort = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  avgIDRShort = sum(tempDf['IDRShort']/len(tempDf))
  idrShort.append(avgIDRShort)
    
plt.plot([y for y in range(70, 91)], idrShort, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Initial Detailed Routing Short Violations (IDRV)")
plt.title('Average IDR Short x Row Utilization (Only fully routable circuits)')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
fdrRuntime = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  fdr = sum(tempDf['FDR']/len(tempDf))
  fdrRuntime.append(fdr)

plt.plot([y for y in range(70, 91)], fdrRuntime, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Runtime (seconds)")
plt.title('Average Runtime to complete routing (Only fully routable circuits)')